In [1]:
from __future__ import print_function

import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import SeparableConv1D, Conv1D, MaxPooling1D
from keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split

import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

save_dir = os.path.join(os.getcwd(), 'saved_models')

from IPython.display import clear_output
from keras_helper import PlotProgress

%matplotlib inline

Using TensorFlow backend.


In [2]:
num_classes = 6

In [3]:
def get_confusion_matrix(Y, Y_hat):
    report = classification_report(
        y_true=Y.argmax(axis=1),
        y_pred=Y_hat.argmax(axis=1),
        labels=list(label_mapping.keys()),
        target_names=list(label_mapping.values())
    )
    return report

In [4]:
class PlotProgress(keras.callbacks.Callback):
    
    def __init__(self, entity='loss'):
        self.entity = entity
        
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('{}'.format(self.entity)))
        self.val_losses.append(logs.get('val_{}'.format(self.entity)))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="{}".format(self.entity))
        plt.plot(self.x, self.val_losses, label="val_{}".format(self.entity))
        plt.legend()
        plt.show();

In [5]:
body_acc_x_train = pd.read_csv('../data/har/train/Inertial_Signals/body_acc_x_train.txt', sep='\s+', header=None)
body_acc_y_train = pd.read_csv('../data/har/train/Inertial_Signals/body_acc_y_train.txt', sep='\s+', header=None)
body_acc_z_train = pd.read_csv('../data/har/train/Inertial_Signals/body_acc_z_train.txt', sep='\s+', header=None)

total_acc_x_train = pd.read_csv('../data/har/train/Inertial_Signals/total_acc_x_train.txt', sep='\s+', header=None)
total_acc_y_train = pd.read_csv('../data/har/train/Inertial_Signals/total_acc_y_train.txt', sep='\s+', header=None)
total_acc_z_train = pd.read_csv('../data/har/train/Inertial_Signals/total_acc_z_train.txt', sep='\s+', header=None)

body_gyro_x_train = pd.read_csv('../data/har/train/Inertial_Signals/body_gyro_x_train.txt', sep='\s+', header=None)
body_gyro_y_train = pd.read_csv('../data/har/train/Inertial_Signals/body_gyro_y_train.txt', sep='\s+', header=None)
body_gyro_z_train = pd.read_csv('../data/har/train/Inertial_Signals/body_gyro_z_train.txt', sep='\s+', header=None)

y_train = pd.read_csv('../data/har/train/y_train.txt', sep='\s+', header=None).values

In [6]:
label_info = pd.read_csv('../data/har/activity_labels.txt', sep='\s+', header=None)
label_mapping = dict()

for index, row_data in label_info.iterrows():
    label_mapping.update({row_data[0] -1 : row_data[1]})

In [7]:
label_mapping

{0: 'WALKING',
 1: 'WALKING_UPSTAIRS',
 2: 'WALKING_DOWNSTAIRS',
 3: 'SITTING',
 4: 'STANDING',
 5: 'LAYING'}

In [ ]:
X_train = np.stack([body_acc_x_train,
                    body_acc_y_train,
                    body_acc_z_train,
                    total_acc_x_train,
                    total_acc_y_train,
                    total_acc_z_train,
                    body_gyro_x_train,
                    body_gyro_y_train,
                    body_gyro_z_train,
                   ], axis=-1)

In [ ]:
encoder = OneHotEncoder(categories='auto')
Y_train = encoder.fit_transform(y_train).toarray()

In [ ]:
X_train = X_train.astype('float32')
Y_train = Y_train.astype('float32')

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, 
                                                      Y_train,
                                                      test_size=0.2,
                                                      random_state=123)

In [ ]:
X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape

In [ ]:
# # Main Model
# kernel_size = 32
# max_pool_size = 3
# dropout_rate = 0.25

# input_sample_size = X_train.shape[1:]

# model = Sequential()
# model.add(Conv1D(30, kernel_size, padding='same', input_shape=input_sample_size))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size= max_pool_size))
# model.add(Dropout(dropout_rate))

# model.add(Conv1D(30, kernel_size, padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size= max_pool_size))
# model.add(Dropout(dropout_rate))

# model.add(Flatten())
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

In [ ]:
kernel_size = 32
max_pool_size = 3
dropout_rate = 0.5

input_sample_size = X_train.shape[1:]

model = Sequential()
model.add(Conv1D(32, kernel_size, padding='same', input_shape=input_sample_size))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size= max_pool_size))
model.add(Dropout(dropout_rate))

model.add(Conv1D(16, kernel_size, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size= max_pool_size))
model.add(Dropout(dropout_rate))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [ ]:
%%time
batch_size = 64
epochs = 100

plot_progress = PlotProgress(entity='acc')

save_path = './keras-saves/_latest.ckpt'
try:
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_valid, Y_valid),
              callbacks=[plot_progress],
              shuffle=True,
         )
except KeyboardInterrupt:
#     model.save(save_path)
    print('\nOutput saved to: "{}./*"'.format(save_path))

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
print("Train Confusion Matrix\n", get_confusion_matrix(Y_train, model.predict(X_train)))
print("Validation Confusion Matrix\n", get_confusion_matrix(Y_valid, model.predict(X_valid)))

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [ ]:
from keras import backend as K
layer_name = 'conv1d_16'
filter_index = 0

layer_output = layer_dict[layer_name].output

In [ ]:
layer_dict[layer_name].get_weights()[0].shape

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, to_file='../model.png')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
grouped_mean_spectrogram = spects.groupby(y_train).apply(lambda group: np.mean([specg for frequencies, times, specg in group], axis=0))

In [ ]:
plt.figure(figsize=(20, 10))
for i in grouped_mean_spectrogram.index:
    plt.subplot(2, 3, i)
#     plt.yticks(frequencies)
    plt.imshow(grouped_mean_spectrogram[i], cmap='plasma')
    plt.title(label_mapping.get(i))

In [ ]:
plt.figure(figsize=(20, 20))

sample = 10
cat = 4

for i, (freqs, times, spec) in enumerate(spects.groupby(y_train).get_group(cat)[:sample]):
    plt.subplot(3, 4, i+1)
    plt.yticks(freqs)
    plt.imshow(spec, cmap='plasma')
    plt.title(label_mapping.get(cat))
